In [2]:
import numpy as np
import psutil
import os

In [3]:
zero = np.array([1, 0], dtype=complex)
one = np.array([0, 1], dtype=complex)

H = (1 / np.sqrt(2)) * np.array([[1, 1],
                                  [1, -1]], dtype=complex)

X = np.array([[0, 1],
              [1, 0]], dtype=complex)

Y = np.array([[0, -1j],
              [1j, 0]], dtype=complex)

Z = np.array([[1, 0],
              [0, -1]], dtype=complex)

def Rx(theta):
    cos = np.cos(theta / 2)
    isin = -1j * np.sin(theta / 2)
    return np.array([[cos, isin],
                     [isin, cos]], dtype=complex)

def Rz(theta):
    exp_plus = np.exp(-1j * theta / 2)
    exp_minus = np.exp(1j * theta / 2)
    return np.array([[exp_plus, 0],
                     [0, exp_minus]], dtype=complex)

def Rzz(theta):
    exp_00 = np.exp(-1j * theta / 2)
    exp_11 = np.exp(-1j * theta / 2)
    exp_01 = np.exp(1j * theta / 2)
    exp_10 = np.exp(1j * theta / 2)

    RZZ = np.array([[exp_00, 0,      0,      0],
                    [0,      exp_01, 0,      0],
                    [0,      0,      exp_10, 0],
                    [0,      0,      0,      exp_11]], dtype=complex)
    return RZZ

In [4]:
def initialize_state(n, state_vector=None):
    if state_vector is not None:
        if len(state_vector) != 2**n:
            raise ValueError("State vector length does not match number of qubits.")
        state = state_vector.reshape([2]*n)
    else:
        state = zero
        for _ in range(n - 1):
            state = np.tensordot(state, zero, axes=0)
    return state

def apply_single_qubit_gate(state, gate, target):
    n_qubits = state.ndim
    state = np.moveaxis(state, target, 0)
    state = np.tensordot(gate, state, axes=([1], [0]))
    state = np.moveaxis(state, 0, target)
    return state

def apply_two_qubit_gate(state, gate, qubit1, qubit2):
    n_qubits = state.ndim
    permute_order = [qubit1, qubit2] + [i for i in range(n_qubits) if i != qubit1 and i != qubit2]
    state = np.transpose(state, permute_order)
    original_shape = state.shape
    state = state.reshape(4, -1)
    state = np.tensordot(gate, state, axes=([1], [0]))
    state = state.reshape([2, 2] + list(original_shape[2:]))
    inverse_permute_order = np.argsort(permute_order)
    state = np.transpose(state, inverse_permute_order)
    return state

def create_cnot_gate():
    CNOT = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 0, 1],
                     [0, 0, 1, 0]], dtype=complex)
    return CNOT

In [5]:
n = 2
state = initialize_state(n)

for i in range(n):
    state = apply_single_qubit_gate(state, H, i)

#CNOT_gate = create_cnot_gate()
#state = apply_two_qubit_gate(state, CNOT_gate, qubit1=0, qubit2=1)

#state = apply_two_qubit_gate(state, CNOT_gate, qubit1=1, qubit2=2)

state_vector = state.reshape(-1)

states = [f"|{bin(i)[2:].zfill(n)}⟩" for i in range(2**n)]

print("Final state amplitudes:")
for idx, amplitude in enumerate(state_vector):
    print(f'{states[idx]}: {amplitude}')

norm = np.sum(np.abs(state_vector)**2)
print(f'\nNormalization check (should be 1): {norm}')


Final state amplitudes:
|00⟩: (0.4999999999999999+0j)
|01⟩: (0.4999999999999999+0j)
|10⟩: (0.4999999999999999+0j)
|11⟩: (0.4999999999999999+0j)

Normalization check (should be 1): 0.9999999999999996


In [6]:
def qaoa_circuit_general(graph_edges, gamma_list, beta_list):
    n = max(max(edge) for edge in graph_edges) + 1  # Number of qubits
    p = len(gamma_list)  # Number of QAOA layers

    state = initialize_state(n)

    # Apply Hadamard gates to all qubits
    for qubit in range(n):
        state = apply_single_qubit_gate(state, H, target=qubit)

    for layer in range(p):
        gamma = gamma_list[layer]
        beta = beta_list[layer]

        # Cost Hamiltonian evolution
        for edge in graph_edges:
            qubit1, qubit2 = edge
            RZZ_gate = Rzz(2 * gamma)
            state = apply_two_qubit_gate(state, RZZ_gate, qubit1, qubit2)

        # Mixer Hamiltonian evolution
        for qubit in range(n):
            Rx_gate = Rx(2 * beta)
            state = apply_single_qubit_gate(state, Rx_gate, target=qubit)

    return state

In [1]:
# Define the edges of the graph (e.g., a triangle)
graph_edges = [(0, 1), (1, 2), (2, 0)]

# QAOA parameters for each layer
gamma_list = [np.pi / 4, np.pi / 4]
beta_list = [np.pi / 8, np.pi / 8]

# Run the QAOA circuit
final_state = qaoa_circuit_general(graph_edges, gamma_list, beta_list)

# Reshape and display the final state
n = 3  # Number of qubits
state_vector = final_state.reshape(-1)
states = [f"|{bin(i)[2:].zfill(n)}⟩" for i in range(2**n)]

print("Final state amplitudes:")
for idx, amplitude in enumerate(state_vector):
    print(f'{states[idx]}: {amplitude} probability {np.abs(amplitude)**2}')

norm = np.sum(np.abs(state_vector)**2)
print(f'\nNormalization check (should be 1): {norm}')


NameError: name 'np' is not defined